In [1]:
!pip install -r requirements.txt

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
user = "cathy"
apikey = {'cathy':"YKLybOknEjqrXePYZPyzPnOro8siaYNfYkcqhRTe"}

In [3]:
import fooddatacentral as fdc
import pandas as pd
import warnings
import numpy as np
import re

In [4]:
fdc_codes = pd.read_csv('hw_fdc_lower.csv')
fdc_codes_multiplier = pd.read_csv('fdc_lower_quant.csv')
hhsize = pd.read_csv('hhcharacteristics_hw.csv')
expenditure = pd.read_csv('expenditures_hw.csv')
consumption = pd.read_csv('hw_consumption_lower.csv')
food_prices = pd.read_csv('ethiopia_food_prices.csv')
consumption = consumption.fillna(0)
fdc_codes.head()

,food item,fdc id
0,pea,788385
1,banana,786652
2,barley,489276
3,cheese,357068
4,coffee,777790


In [5]:
fdc_codes

,food item,fdc id
0,pea,788385
1,banana,786652
2,barley,489276
3,cheese,357068
4,coffee,777790
5,meat,548512
6,butter,479107
7,chickpeas,393415
8,eggs,520005
9,haricot_beans,747441


In [6]:
fdc.search(apikey[user],"lettuce").head()

,additionalDescriptions,allHighlightFields,dataType,description,fdcId,foodCode,foodNutrients,ndbNumber,publishedDate,scientificName,score
0,"lettuce, NFS; iceberg","<b>Includes</b>: <em>lettuce</em>, NFS; iceberg",Survey (FNDDS),"Lettuce, raw",787797,75113000,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,2020-04-01,NaN,465.89352
1,NaN,,Survey (FNDDS),"Lettuce, manoa",787799,75113070,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,2020-04-01,NaN,452.89624
2,Bibb; deer tongue lettuce; native lettuce; red...,<b>Includes</b>: Bibb; deer tongue <em>lettuce...,Survey (FNDDS),"Lettuce, Boston, raw",787798,75113060,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,2020-04-01,NaN,421.18920
3,NaN,,Survey (FNDDS),"Lettuce, arugula, raw",787800,75113080,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,2020-04-01,NaN,406.42944
4,endive; chicory; escarole,,Survey (FNDDS),"Romaine lettuce, raw",787271,72116000,"[{'nutrientId': 1003, 'nutrientName': 'Protein...",NaN,2020-04-01,NaN,406.42944


## Nutrition By Food

In [7]:
def nutrients_per_food(i):
    food = fdc_codes.iloc[i,:][0]
    code = fdc_codes.iloc[i,:][1]
    temp = fdc.nutrients(apikey[user],fdc_id=code)
    temp = temp.drop(columns=['Units'])
    temp.columns = [food]
    return temp

In [8]:
fdc.nutrients(apikey[user],fdc_id=357068)

,Quantity,Units
"Calcium, Ca",536.00,mg
"Carbohydrate, by difference",3.57,g
Cholesterol,54.00,mg
Energy,250.00,kcal
"Fatty acids, total saturated",12.50,g
"Fiber, total dietary",0.00,g
"Iron, Fe",0.00,mg
Protein,17.86,g
"Sodium, Na",1286.00,mg
"Sugars, total including NLEA",3.57,g


In [9]:
merged = nutrients_per_food(0)
for i in np.arange(1, len(fdc_codes)):
    new = nutrients_per_food(i)
    merged = merged.merge(new, left_index=True, right_index=True, how = 'outer')
merged = merged.fillna(0)

In [10]:
merged

,pea,banana,barley,cheese,coffee,meat,butter,chickpeas,eggs,haricot_beans,...,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat,lettuce
10:0,0.000,0.001,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.000,0.00,0.001,0.0,0.000,0.0,0.00,0.000,0.000
12:0,0.000,0.002,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.003,0.00,0.003,0.0,0.001,0.0,0.00,0.000,0.000
14:0,0.002,0.002,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.000,0.00,0.001,0.0,0.002,0.0,0.00,0.000,0.000
15:0,0.000,0.000,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.000,0.00,0.000,0.0,0.001,0.0,0.00,0.000,0.000
16:0,0.139,0.102,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.528,0.00,0.016,0.0,0.543,0.0,0.00,0.000,0.016
16:1,0.002,0.010,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.014,0.00,0.001,0.0,0.016,0.0,0.00,0.000,0.001
16:1 t,0.000,0.000,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.000,0.00,0.000,0.0,0.002,0.0,0.00,0.000,0.000
17:0,0.000,0.000,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.000,0.00,0.000,0.0,0.002,0.0,0.00,0.000,0.000
18:0,0.022,0.005,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.154,0.00,0.004,0.0,0.052,0.0,0.00,0.000,0.002
18:1,0.035,0.022,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,...,0.00,0.739,0.00,0.001,0.0,1.101,0.0,0.00,0.000,0.004


## 

In [11]:
hhsize = hhsize.drop(columns = ['t'])
hhsize.head()

,j,M 0-3,M 4-8,M 9-13,M 14-18,M 19-30,M 31-50,M 51+,F 0-3,F 4-8,F 9-13,F 14-18,F 19-30,F 31-50,F 51+
0,1.010102e+12,0,1,1,1,0,1,0,0,0,1,0,0,1,1
1,1.010102e+12,0,0,2,0,0,1,0,0,1,1,0,0,1,0
2,1.010102e+12,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1.010102e+12,0,0,0,0,0,0,0,0,0,2,0,1,0,1
4,1.010102e+12,0,0,0,0,0,0,1,0,0,1,1,0,1,0


In [12]:
nmin = pd.read_csv('./diet_minimums.csv').set_index('Nutrition')
girl = nmin["F 4-8"]
boys = nmin["M 4-8"]
women = nmin["F 31-50"]
men = nmin["M 31-50"]

In [13]:
def hh_nutrition(i):
    skeleton = nmin.copy()
    skeleton = skeleton.drop(skeleton.columns[:], axis = 1)
    #count = hhsize.iloc[i,:][1:].values
    groups = hhsize.loc[i, :].index[1:]
    values = hhsize.loc[i, :].values[1:]
    total = 0
    for i in np.arange(len(groups)):
        if groups[i] == 'M 0-3' or groups[i] == 'F 0-3':
            total += nmin['C 1-3'].values * values[i] 
        else:
            total += nmin[groups[i]].values * values[i] 
    skeleton[hhsize.iloc[i,:][0]] = total
    return skeleton

In [14]:
hh_nutrition(0)

,1010201403011.0
Nutrition,
Energy,12600.0
Protein,287.0
"Fiber, total dietary",176.4
"Folate, DFE",2400.0
"Calcium, Ca",8100.0
"Carbohydrate, by difference",910.0
"Iron, Fe",71.0
"Magnesium, Mg",2080.0
Niacin,92.0


In [15]:
def meets(consumed, needed):
    return sum(consumed - needed)

In [16]:
expenditure.head()

,t,j,Field Pea,Banana,Barley,Beef,Beer,Boye/Yam,Bula,Butter/ghee,...,Salt,Soft drinks/Soda,Sorghum,Sugar,Sweet potato,Tea,Teff,Tella,Tomato,Wheat
0,2011,1.010102e+12,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,1.00,NaN,0.0,18.0,NaN,NaN,0.0,NaN,NaN,0.0
1,2011,1.010102e+12,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.50,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
2,2011,1.010102e+12,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.50,NaN,0.0,5.0,NaN,NaN,0.0,NaN,NaN,0.0
3,2011,1.010102e+12,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.50,NaN,84.0,18.0,NaN,NaN,0.0,NaN,NaN,0.0
4,2011,1.010102e+12,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0.05,NaN,0.0,18.0,NaN,NaN,0.0,NaN,NaN,0.0


In [17]:
consumption.head()

,t,j,u,Field Pea,banana,barley,Beef,Beer,Boye/Yam,bula,...,salt,Soft drinks/Soda,sorghum,sugar,Sweet potato,Tea,teff,Tella,Tomato,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2011,1.010102e+12,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,125.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
consumption = consumption.drop(['Chat / Kat'], axis = 1)
consumption = consumption.drop(['bula'], axis = 1)
consumption = consumption.drop(['kocho'], axis = 1)
consumption = consumption.drop(['niger_seed'], axis = 1)

In [19]:
dropping = []
total = 0
for i in np.arange(len(consumption)):
    if sum(consumption.iloc[i,:][3:].values) == 0:
        dropping.append(i)
    total += 1
consumption = consumption.drop(dropping)

In [20]:
unconsumed = []
run = 0
for i in consumption.columns.values[3:]:
    test = consumption.filter(items=[i])
    if len(test.loc[(test != 0).all(axis=1), :]) == 0:
        unconsumed.append(i)
    run += 1
unconsumed
consumption = consumption.drop(unconsumed, axis = 1)
consumption.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,125.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
6,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0


In [21]:
consumption = consumption.replace('Araba', 'Gram')
consumption = consumption.replace('40', 'Gram')
#consumption = consumption.replace(0, 'Gram')
consumption = consumption.replace('Cm', 'Gram')
consumption = consumption.replace('Cubic Centimeter', 'Gram')
consumption = consumption.replace('Tuba', 'Gram')
consumption = consumption.replace('Box', 'Gram')
consumption = consumption.replace('Other(Specify)', 'Gram')
consumption = consumption.replace('Shekim', 'Gram')

In [22]:
consumption.u.unique()

array(['Gram', 'Kg', 'Number', 'Liter', 'Cup', 'Pack', 'Roll', 'Pair', 0],
      dtype=object)

In [23]:
consumption = consumption[(consumption['u'] != 'Pair')]
consumption = consumption[(consumption['u'] != 'Roll')]
consumption = consumption[(consumption['u'] != 'Pack')]
consumption = consumption[(consumption['u'] != 0)]

In [24]:
consumption.u.unique()

array(['Gram', 'Kg', 'Number', 'Liter', 'Cup'], dtype=object)

In [25]:
consumption.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,125.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
6,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0


In [26]:
eleven = consumption[consumption['t'] == 2011]
thir = consumption[(consumption['t'] == 2013) & (consumption['j'] != 0)]
eleven.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,125.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
6,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0


In [27]:
thir.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
11140,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,250.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11141,2013,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
11142,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11144,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,500.0,0.0,250.0,0.0,0.0,0.0,0.0
11145,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
eleven_no_num = eleven[eleven['u'] != 'Number']
thir_no_num = thir[thir['u'] != 'Number']

## 2011 Data

In [29]:
eleven_no_num.u.unique()

array(['Gram', 'Kg', 'Liter', 'Cup'], dtype=object)

In [30]:
def convert_kg(row):
    for i in np.arange(len(row.values[3:])):
        if row.values[i+3] != 0:
            row.values[i+3] = row.values[i+3]*1000
    return row

In [31]:
def convert_l(row):
    for i in np.arange(len(row.values[3:])):
        if row.values[i+3] != 0:
            row.values[i+3] = row.values[i+3]*1000
    return row

In [32]:
def convert_cup(row):
    for i in np.arange(len(row.values[3:])):
        if row.values[i+3] != 0:
            row.values[i+3] = row.values[i+3]*128
    return row

In [33]:
eleven_no_num.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,125.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
6,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0


In [34]:
eleven_no_num.apply(lambda x: x*1000 if x['u'] == 'Kg' and type(x) == int else x, axis = 1)
eleven_no_num.apply(lambda x: x*1000 if x['u'] == 'Liter' and type(x) == int else x, axis = 1)
eleven_no_num.apply(lambda x: x*128 if x['u'] == 'Cup' and type(x) == int else x, axis = 1)
eleven_no_num.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,125.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
6,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0


In [35]:
eleven_no_num = eleven_no_num.groupby(['j']).sum()
eleven_no_num.head()

,t,banana,barley,cheese,chickpeas,coffee,eggs,pea,haricot_beans,horsebeans,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
j,,,,,,,,,,,,,,,,,,,,,
1.010102e+12,4022,0.0,0.0,0.0,0.0,125.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.00,0.0,250.0,0.0,1.00,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.50,0.0,125.0,0.0,0.00,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.25,0.0,125.0,4.0,0.25,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,250.00,0.0,0.0,0.0,750.0,...,0.0,0.0,0.0,1.00,0.0,125.0,21.0,1.00,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,125.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.00,0.0,125.0,0.0,1.00,0.0,0.0


In [36]:
def multiplied_n(hid, df):
    row = df.loc[hid]
    nutr_skeleton = merged.copy()
    nutr_skeleton = nutr_skeleton.drop(nutr_skeleton.columns[:], axis = 1)
    total = 0
    for i in np.arange(len(row[3:])):
        foods = row.index[3:]
        amounts = row.values[3:]
        total += merged[foods[i]]*amounts[i]/100
    return total.to_frame()

In [37]:
def met(index, df):
    t = hh_nutrition(index)
    u = multiplied_n(t.columns[0], df)
    new = t.join(u, how = 'inner')
    if sum(new.iloc[:,1] - new.iloc[:,0]) < 0:
        return False
    else:
        return True

In [93]:
def met_vitA(index, df):
    t = hh_nutrition(index)
    u = multiplied_n(t.columns[0], df)
    new = t.join(u, how = 'inner')
    if new.iloc[13,1] - new.iloc[13,0] < 0:
        return False
    else:
        return True

In [102]:
hello = hh_nutrition(1)
ejhe = multiplied_n(hello.columns[0], eleven_no_num)
new = hello.join(ejhe, how = 'inner')
new

,1010201403011.0,cheese
Energy,10400.0,0.0
Protein,223.0,0.0
"Fiber, total dietary",145.6,0.0
"Folate, DFE",1900.0,0.0
"Calcium, Ca",6900.0,0.0
"Carbohydrate, by difference",780.0,0.0
"Iron, Fe",60.0,0.0
"Magnesium, Mg",1590.0,0.0
Niacin,74.0,0.0
"Phosphorus, P",5650.0,0.0


In [41]:
eleven_no_num.head()

,t,banana,barley,cheese,chickpeas,coffee,eggs,pea,haricot_beans,horsebeans,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
j,,,,,,,,,,,,,,,,,,,,,
1.010102e+12,4022,0.0,0.0,0.0,0.0,125.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.00,0.0,250.0,0.0,1.00,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.50,0.0,125.0,0.0,0.00,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.25,0.0,125.0,4.0,0.25,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,250.00,0.0,0.0,0.0,750.0,...,0.0,0.0,0.0,1.00,0.0,125.0,21.0,1.00,0.0,0.0
1.010102e+12,4022,0.0,0.0,0.0,0.0,125.00,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.00,0.0,125.0,0.0,1.00,0.0,0.0


In [42]:
t = hh_nutrition(0)
u = multiplied_n(t.columns[0], eleven_no_num)
new = t.join(u, how = 'inner')

In [43]:
total_met = 0
total = 0
for i in np.arange(len(hhsize)):
    if met(i, eleven_no_num):
        total_met += 1
    total +=1
total

7746

In [94]:
total_metA = 0
totalA = 0
for i in np.arange(len(hhsize)):
    if met_vitA(i, eleven_no_num):
        total_metA += 1
    totalA +=1
totalA

7746

In [95]:
total_metA

54

## 2013 Data

In [46]:
thir_no_num.u.unique()

array(['Gram', 'Kg', 'Liter', 'Cup'], dtype=object)

In [47]:
thir_no_num.apply(lambda x: x*1000 if x['u'] == 'Kg' and type(x) == int else x, axis = 1)
thir_no_num.apply(lambda x: x*1000 if x['u'] == 'Liter' and type(x) == int else x, axis = 1)
thir_no_num.apply(lambda x: x*128 if x['u'] == 'Cup' and type(x) == int else x, axis = 1)
thir_no_num = thir_no_num.groupby(['j']).sum()
thir_no_num.head()

,t,banana,barley,cheese,chickpeas,coffee,eggs,pea,haricot_beans,horsebeans,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
j,,,,,,,,,,,,,,,,,,,,,
1.010102e+12,6039,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,500.0,...,2.5,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1.010102e+12,4026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,500.0,0.0,0.0,500.0,0.0,250.0,0.0,0.0,0.0,0.0
1.010102e+12,4026,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,500.0,...,0.0,0.0,0.0,500.0,0.0,100.0,5.0,500.0,0.0,0.0
1.010102e+12,4026,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,500.0,...,0.0,0.0,0.0,1.0,0.0,250.0,20.0,500.0,0.0,0.0
1.010102e+12,4026,0.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,...,500.0,0.0,1.0,500.0,0.0,250.0,5.0,1.0,0.0,0.0


In [48]:
thir.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
11140,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,250.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11141,2013,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
11142,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11144,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,500.0,0.0,250.0,0.0,0.0,0.0,0.0
11145,2013,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
total_met_2013 = 0
total_2013 = 0
for i in np.arange(len(hhsize)):
    if met(i, thir_no_num):
        total_met_2013 += 1
    total_2013 +=1
total_2013

7746

## Random Tests

In [51]:
# weird_values = consumption[(consumption['u'] == 'Pack') | (consumption['u'] == 'Roll') | (consumption['u'] == 'Pair')]
# weird_values.head()

In [52]:
# dropped_hh = []
# for i in np.arange(len(weird_values)):
#     new_hh = weird_values.iloc[i,1]
#     if new_hh not in dropped_hh:
#         dropped_hh.append(new_hh)

In [53]:
#consumption = consumption[~consumption.isin(dropped_hh)]

In [54]:
#nan = consumption[consumption['u'] == 'Pack'].iloc[0,:][1]
#consumption = consumption[consumption['u'] != nan]

In [55]:
#for i in np.arange(len(consumption)):
    #if consumption.iloc[i,1] in dropped_hh:
        #consumption = consumption.drop(consumption.index[i])

In [56]:
foods = fdc_codes.copy()
foods = foods.drop(columns = ["fdc id"])
foods.head()

,food item
0,pea
1,banana
2,barley
3,cheese
4,coffee


In [57]:
# measurements = []
# for i in foods.values:
#     temp = consumption.filter(items=['u',i[0]])
#     measurement = temp.loc[(temp != 0).all(axis=1), :].u.unique()
#     measurements.append(measurement)
# measurements

In [58]:
# max_m = 0
# for i in measurement:
#     if len(i) > max_m:
#         max_m = len(i)

In [59]:
# foods['measurements'] = measurements
# foods

In [60]:
food_prices

,Unnamed: 0,Food,Units,Price,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,0,Banana_mean,lb,16.000000,NaN,,NaN
1,1,Beef_mean,lb,146.000000,NaN,,NaN
2,2,Butter_mean,lb,109.000000,NaN,,NaN
3,3,eggs_mean,12,63.000000,NaN,,NaN
4,4,lettuce_mean,head,22.000000,NaN,,NaN
5,5,onion_mean,lb,12.000000,NaN,,NaN
6,6,potato_mean,lb,16.000000,NaN,,NaN
7,7,tomato_mean,lb,11.000000,NaN,,NaN
8,8,oranges_mean,lb,2.000000,NaN,,NaN
9,9,pea_mean,kg,10.567551,NaN,,NaN


In [61]:
food_prices.replace('_mean','',regex=True, inplace = True)
food_prices['Food'] = food_prices['Food'].str.lower()

In [62]:
food_prices.head()

,Unnamed: 0,Food,Units,Price,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,0,banana,lb,16.0,NaN,,NaN
1,1,beef,lb,146.0,NaN,,NaN
2,2,butter,lb,109.0,NaN,,NaN
3,3,eggs,12,63.0,NaN,,NaN
4,4,lettuce,head,22.0,NaN,,NaN


In [63]:
food_prices.sort_values('Food')

,Unnamed: 0,Food,Units,Price,Unnamed: 4,Unnamed: 5,Unnamed: 6
9,9,pea,kg,10.567551,NaN,,NaN
1,1,beef,lb,146.000000,NaN,,NaN
3,3,eggs,12,63.000000,NaN,,NaN
13,13,goat,goat,986.205882,NaN,,NaN
17,17,milk,L,30.355556,NaN,,NaN
20,20,teff,100kg,1141.469693,NaN,,NaN
15,15,maize,kg,6.569940,NaN,,NaN
5,5,onion,lb,12.000000,NaN,,NaN
19,19,sugar,kg,20.564407,NaN,,NaN
21,21,wheat,kg,9.433715,NaN,,NaN


In [64]:
merged.head()

,pea,banana,barley,cheese,coffee,meat,butter,chickpeas,eggs,haricot_beans,...,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat,lettuce
10:0,0.000,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.001,0.0,0.000,0.0,0.0,0.0,0.000
12:0,0.000,0.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.003,0.0,0.003,0.0,0.001,0.0,0.0,0.0,0.000
14:0,0.002,0.002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.001,0.0,0.002,0.0,0.0,0.0,0.000
15:0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000,0.0,0.000,0.0,0.001,0.0,0.0,0.0,0.000
16:0,0.139,0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.528,0.0,0.016,0.0,0.543,0.0,0.0,0.0,0.016


In [65]:
consumption.head()

,t,j,u,banana,barley,cheese,chickpeas,coffee,eggs,pea,...,meat,milk,millet,onion,potato,salt,sorghum,sugar,teff,wheat
0,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,125.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0
1,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0
4,2011,1.010102e+12,Kg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
6,2011,1.010102e+12,Gram,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,125.0,0.0,0.0,0.0,0.0


In [66]:
hhsize.head()

,j,M 0-3,M 4-8,M 9-13,M 14-18,M 19-30,M 31-50,M 51+,F 0-3,F 4-8,F 9-13,F 14-18,F 19-30,F 31-50,F 51+
0,1.010102e+12,0,1,1,1,0,1,0,0,0,1,0,0,1,1
1,1.010102e+12,0,0,2,0,0,1,0,0,1,1,0,0,1,0
2,1.010102e+12,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1.010102e+12,0,0,0,0,0,0,0,0,0,2,0,1,0,1
4,1.010102e+12,0,0,0,0,0,0,1,0,0,1,1,0,1,0
